In [1]:

import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubun

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-08-23 03:59:20--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.4’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.09s   

2021-08-23 03:59:20 (11.1 MB/s) - ‘postgresql-42.2.16.jar.4’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Software_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   17747349|R2EI7QLPK4LF7U|B00U7LCE6A|     106182406|CCleaner Free [Do...|Digital_Software|          4|            0|          0|   N|                Y|          Four Stars|      So far so good| 2015-08-31|
|         US|   10956619|R1W5OMFK1Q3I3O|B00HRJMOM4|     162269768|ResumeMaker Profe...|Digital_Software|          3|    

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2EI7QLPK4LF7U|          4|            0|          0|   N|                Y|
|R1W5OMFK1Q3I3O|          3|            0|          0|   N|                Y|
| RPZWSYWRP92GI|          1|            1|          2|   N|                Y|
|R2WQWM04XHD9US|          5|            0|          0|   N|                Y|
|R1WSPK2RA2PDEF|          4|            1|          2|   N|                Y|
|R11JVGRZRHTDAS|          5|            4|          4|   N|                N|
|R2B8468OKXXYE2|          1|            1|          1|   N|                N|
|R2HGGCCZSSNUCB|          1|            0|          0|   N|                Y|
| REEE4LHSVPRV9|          1|            0|          0|   N|                Y|
|R25OMUUILFFHI9|          5|            0|          0|   N|     

In [6]:
#filter by total_votes
reviews_df = vine_df.filter("total_votes >= 20")
reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RXKYQDUACCRIA|          2|           20|         20|   N|                N|
|R10Q403A12Q9PX|          1|          123|        136|   N|                N|
|R2OH6106YSNSV4|          1|           20|         24|   N|                Y|
|R3C1OKM81KRDMO|          4|          159|        167|   N|                Y|
|R180C1J73S8MCV|          1|           27|         31|   N|                N|
|R208352XIVG6CL|          2|           17|         25|   N|                N|
|R1J4DGLB9OKW2B|          4|            9|         20|   N|                N|
|R2G3JWLCDAPN5I|          5|            9|         21|   N|                N|
| RXM58ZCC2N93C|          1|           17|         23|   N|                N|
| RJSGOYKD6T7KY|          5|            3|         28|   N|     

In [7]:
#Check Schema for datatypes
reviews_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)



In [8]:
# Create a new column with vote ratio calculated
ratio_df = reviews_df.withColumn('vote_ratio', (reviews_df['helpful_votes']/reviews_df['total_votes']))
ratio_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|         vote_ratio|
+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
| RXKYQDUACCRIA|          2|           20|         20|   N|                N|                1.0|
|R10Q403A12Q9PX|          1|          123|        136|   N|                N| 0.9044117647058824|
|R2OH6106YSNSV4|          1|           20|         24|   N|                Y| 0.8333333333333334|
|R3C1OKM81KRDMO|          4|          159|        167|   N|                Y| 0.9520958083832335|
|R180C1J73S8MCV|          1|           27|         31|   N|                N| 0.8709677419354839|
|R208352XIVG6CL|          2|           17|         25|   N|                N|               0.68|
|R1J4DGLB9OKW2B|          4|            9|         20|   N|                N|               0.45|
|R2G3JWLCDAPN5I|    

In [9]:
#filter by vote ratio greater than .5
top_votes_df = ratio_df.filter("vote_ratio >= .5")
top_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|        vote_ratio|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| RXKYQDUACCRIA|          2|           20|         20|   N|                N|               1.0|
|R10Q403A12Q9PX|          1|          123|        136|   N|                N|0.9044117647058824|
|R2OH6106YSNSV4|          1|           20|         24|   N|                Y|0.8333333333333334|
|R3C1OKM81KRDMO|          4|          159|        167|   N|                Y|0.9520958083832335|
|R180C1J73S8MCV|          1|           27|         31|   N|                N|0.8709677419354839|
|R208352XIVG6CL|          2|           17|         25|   N|                N|              0.68|
| RXM58ZCC2N93C|          1|           17|         23|   N|                N|0.7391304347826086|
| RC9E2K8LLD6P6|          1|  

In [10]:
# Yes Votes
yes_df = top_votes_df.filter(top_votes_df["vine"] == "Y")
yes_df.show()

+---------+-----------+-------------+-----------+----+-----------------+----------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|vote_ratio|
+---------+-----------+-------------+-----------+----+-----------------+----------+
+---------+-----------+-------------+-----------+----+-----------------+----------+



In [11]:
# No Votes
no_df = top_votes_df.filter(top_votes_df["vine"] == "N")
no_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|        vote_ratio|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| RXKYQDUACCRIA|          2|           20|         20|   N|                N|               1.0|
|R10Q403A12Q9PX|          1|          123|        136|   N|                N|0.9044117647058824|
|R2OH6106YSNSV4|          1|           20|         24|   N|                Y|0.8333333333333334|
|R3C1OKM81KRDMO|          4|          159|        167|   N|                Y|0.9520958083832335|
|R180C1J73S8MCV|          1|           27|         31|   N|                N|0.8709677419354839|
|R208352XIVG6CL|          2|           17|         25|   N|                N|              0.68|
| RXM58ZCC2N93C|          1|           17|         23|   N|                N|0.7391304347826086|
| RC9E2K8LLD6P6|          1|  

In [12]:
# Load Library
from pyspark.sql.functions import round, col

# define function to evaluate divisor
def safe_div(x, y):
    return 0 if y == 0 else x / y

In [13]:
# extracting summary values from the no_df dataframe
nreview_count = no_df.count()
nFiveStar_total = no_df.filter('star_rating == 5').count()
nFiveStar_Per = safe_div(nFiveStar_total, nreview_count)
npercentage = "{:.0%}".format(nFiveStar_Per)

In [14]:
# extracting summary values from the yes_df dataframe
yreview_total = yes_df.count()
yFiveStar_total = yes_df.filter('star_rating == 5').count()
yFiveStar_Per = safe_div(yFiveStar_total, yreview_total)
ypercentage = "{:.0%}".format(yFiveStar_Per)

In [15]:
# printing

print(f'Reviewers in the Vine program')
print(f'-----------------------------')
print(f'Total Reviews: {yreview_total}')
print(f'Number of 5-star reviews: {yFiveStar_total}')
print(f'Percentage: {ypercentage}')

print(f'\n\nReviewers not in the Vine program')
print(f'---------------------------------')
print(f'Total Reviews: {nreview_count}')
print(f'Number of 5-star reviews: {nFiveStar_total}')
print(f'Percentage: {npercentage}')

Reviewers in the Vine program
-----------------------------
Total Reviews: 0
Number of 5-star reviews: 0
Percentage: 0%


Reviewers not in the Vine program
---------------------------------
Total Reviews: 2120
Number of 5-star reviews: 314
Percentage: 15%
